In [2]:
##数据库配置、连接、关闭-基本操作

from peewee import *

# SQLite database using WAL journal mode and 64MB cache.
sqlite_db = SqliteDatabase('sqlite_db.db',   # 文件名（当前目录），路径（其他文件夹）
                           pragmas={  # 各种配置
                              'journal_mode': 'wal',
                              'cache_size': -1024 * 64})
sqlite_db.connect()
sqlite_db.close()

True

In [4]:
## SQLite的扩展 SqliteDatabase
#                   --> SqliteExtDatabase 
#                       --> PooledSqliteExtDatabase    
#                   --> PooledSqliteDatabase
# 这4个都可以建立连接

from playhouse.sqlite_ext import SqliteExtDatabase

# Use SQLite (will register a REGEXP function and set busy timeout to 3s).
db = SqliteExtDatabase('people.db',
                       regexp_function=True,
                       timeout = 4,
                       pragmas = {
                           'journal_mode': 'wal',
                       } )


In [5]:
'''
db.pragma() 配置属性
    - cache_size: -1 * 数据大小kb  
    - journal_mode: 
        - wal
    - foreign_keys:
    - ignore_check_constrains：
    - synchronous：  
    - 以上是常用的

# db.cache_size 调用属性 
'''
db.pragma('cache_size', -1024 *64) # permanent= True 强制持久化

In [16]:
print('cache_size:', db.cache_size)
print('foreign_keys:', db.foreign_keys) 
print('journal_mode:', db.journal_mode)
print('page_size:', db.page_size)
print(db._table_functions)

cache_size: -65536
foreign_keys: 0
journal_mode: wal
page_size: 4096
[]


In [15]:
db.pragma('foreign_keys', 0, permanent= True)

In [17]:
#用户自定义函数部分没有看懂，装饰器的用法也忘记了

In [22]:
# 在程序运行时设置数据库的三种方法
#1 
db_1 = SqliteDatabase('my_app.db', pragmas = {'journal_mode':'wal'})

#2
db_2 = SqliteDatabase(None)

db_2.init('de_filename.db', pragmas={'journal_mode':'wal'})

#3
db_3 = DatabaseProxy()

db_3.initialize(db_1)


In [24]:
db_1.connect()

OperationalError: Connection already opened.

In [25]:
db_1.connect(reuse_if_open=True)

False

In [27]:
db_1.close()

False

In [28]:
db_2.is_closed

<bound method Database.is_closed of <peewee.SqliteDatabase object at 0x00000186F22BAB80>>

In [30]:
db_2.connect(reuse_if_open=True)

False

In [35]:
db_2.is_closed()

True

In [34]:
with db_2:
    print(db.is_closed())

False


In [32]:
db_2.close()

True

In [39]:
with db_1.connection_context():
    print(db_1.is_connection_usable())
    print(db_1.is_closed())

True
False


In [40]:
# 等下认真学习下装饰器的玩法，这里基本都在用

In [41]:
from playhouse.pool import PooledSqliteDatabase

db = PooledSqliteDatabase(
    'my_app.db', 
    max_connections= 30, # 最大连接数
    stale_timeout= 300, # 过期时间
    pragmas = {  # 其他参数
        'journal_mode': 'wal'
    })

class BaseModel(Model):
    class Meta:
        database = db

In [45]:
import unittest
from peewee import *

test_db = SqliteDatabase(':memory:')

class User(BaseModel):
    pass

class Tweet(BaseModel):
    pass

class EventLog(BaseModel):
    pass

class Relationship(BaseModel):
    pass

MODELS = [User, Tweet, EventLog, Relationship]

class BaseTestCase(unittest.TestCase):
    def setup(self):
        test_db.bind(# 3个参数
            MODELS,  # 准备绑定的Models
            bind_refs=False,  # 绑定引用的模型
            bind_backrefs=False) # 绑定反向引用的模型
        test_db.connect() 
        test_db.create_tables(MODELS)

    def tearDown(self):
        test_db.drop_tables(MODELS)
        test_db.close()

test_db.bind()

x = BaseTestCase()
x.setup()
x.tearDown()




In [46]:
# 与flask框架完美集成

from flask import Flask
from peewee import *

database = SqliteDatabase('my_app.db')
app = Flask(__name__)

# This hook ensures that a connection is opened to handle any queries
# generated by the request.
@app.before_request
def _db_connect():
    database.connect()

# This hook ensures that the connection is closed when we've finished
# processing the request.
@app.teardown_request
def _db_close(exc):
    if not database.is_closed():
        database.close()

In [6]:
'''
装饰器知识复习
'''

def hi(name = 'yasoob'):
    return 'hi ' + name

print(hi())

hi yasoob


In [9]:
# greet = hi
print(greet())

hi yasoob


In [8]:
del hi
print(hi())

NameError: name 'hi' is not defined

In [10]:
print(greet())

hi yasoob


In [12]:
def hi(name="yasoob"):
    def greet():
        return "now you are in the greet() function"

    def welcome():
        return "now you are in the welcome() function"

    if name == "yasoob":
        return greet
    else:
        return welcome

a = hi()
print(a)
print(a())

<function hi.<locals>.greet at 0x000001EA5CD6B550>
now you are in the greet() function


In [13]:
def hi():
    return 'hi yasoob!'

def do(func):
    print('xxxx')
    print(func())

do(hi)

xxxx
hi yasoob!


In [17]:
def hi(name = 'yasoob'):
    print('now you are inside the hi()')

    def greet():
        return 'in greet()'

    def welcome():
        return 'in welcome()'

    print(greet())
    print(welcome())
    print('in hi()')

hi()

greet()
welcome()

now you are inside the hi()
in greet()
in welcome()
in hi()


NameError: name 'welcome' is not defined

In [21]:
def a_new_decorator(a_func):

    def warpTheFunction():
        print('前a_func()')

        a_func()

        print('a_func后')
    
    return warpTheFunction


def a_func_requiring_decoration():
    print('我就是那个被装饰的函数')



a_func_requiring_decoration()

a_func_requiring_decoration = a_new_decorator(a_func_requiring_decoration)

a_func_requiring_decoration()



我就是那个被装饰的函数
前a_func()
我就是那个被装饰的函数
a_func后


In [22]:
@a_new_decorator
def another_func():
    """Hey you! Decorate me"""
    print('我是另外一个被装饰的函数')

another_func()

前a_func()
我是另外一个被装饰的函数
a_func后


In [25]:
print(another_func.__doc__)
print(another_func.__name__)

None
warpTheFunction


In [27]:
from functools import wraps

def a_new_decorator(a_func):
    @wraps(a_func)
    def warpTheFunction():
        print('前')
        a_func()
        print('后')

    return warpTheFunction

@a_new_decorator
def another_func():
    """Hey you! Decorate me"""
    print('我是另外一个被装饰的函数')

print(another_func.__name__)
print(another_func.__doc__)





another_func
Hey you! Decorate me


In [29]:
# 装饰器的标准写法

from functools import wraps

def decorator(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated

@decorator
def func():
    return "Function is running"

can_run = True
func()

'Function is running'

In [30]:
can_run = False
func()

'Function will not run'

In [33]:
print(func.__name__)
print(func.__getattribute__)
# print(func.___)

func
<method-wrapper '__getattribute__' of function object at 0x000001EA5DB5D310>


In [34]:
# flask 授权场景

from functools import wraps

def requires_auth(f):
    @warps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not chech_auth(auth.name, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated



In [39]:
# 日志装饰

from functools import wraps

def logit(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + ' was called')
        return func(*args, **kwargs)
        
    return with_logging

@logit
def a_func_requiring_decoration():
    print('我就是那个被装饰的函数')

a_func_requiring_decoration()

a_func_requiring_decoration was called
我就是那个被装饰的函数


In [43]:
from functools import wraps

def logit(logfile = 'out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_str = func.__name__ + ' was called'
            print(log_str)
            with open(logfile, 'a') as open_file:
                open_file.write(log_str +'\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator

@logit()
def func1():
    pass

@logit(logfile='func2.log')
def func2():
    pass

func1()
func2()



func1 was called
func2 was called


In [56]:

from functools import wraps

class logit(object):
    def __init__(self, logfile = 'out.log'):
        self.logfile = logfile

    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_str = func.__name__ +' was called'
            print(log_str)
            with open(self.logfile, 'a') as open_file:
                open_file.write(log_str + '\n')
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function

    def notify(self):
        pass

@logit()
def myfunc1():
    pass

myfunc1()

class email_logit(logit):
    def __init__(self, email = 'admin@project.com', *args, **kwargs):
        self.email = email
        super(email_logit, self).__init(*args, **kwargs)

    def notify(self):

        pass




myfunc1 was called
